In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# DCGAN= Deep Convolution GANs
Upsampling convolution
It is also called often.
`Deconvolution or 
Fractionally strided convolution or
Upconvolution`

# Output Size= (I-1)*S-2P+K
I=Input size
S=Stride
P=Padding
K=Kernel


In [2]:
import torch
import torch.nn as nn

input=torch.randn(1,1,6,6)
input

model=nn.Sequential(
    nn.Conv2d(in_channels=1,
             out_channels=10,
             kernel_size=2,
             padding=1,
             stride=1,
             bias=True),
    nn.LeakyReLU(0.3),
    nn.BatchNorm2d(10),
    nn.Dropout(0.3),
    nn.Tanh()
    
)

output=model(input)
output.shape
# output

torch.Size([1, 10, 7, 7])

In [3]:
import torch 
import torch.nn as nn

input= torch.randn(1,3,8,8)
input

model=nn.Sequential(
    nn.ConvTranspose2d(
        in_channels=3,
        out_channels=1,
        kernel_size=2,
             padding=1,
             stride=1,
             bias=True
        
    ),
    nn.LeakyReLU(0.3),
    nn.BatchNorm2d(1),
    nn.Dropout(0.3),
    nn.Tanh()
)

output=model(input)
output.shape

output


tensor([[[[-0.7836, -0.6287,  0.9983,  1.0000,  0.9323, -0.0000, -0.3069],
          [-0.8856,  0.0000,  0.9573, -0.0000, -0.5525, -0.5878, -0.4065],
          [ 0.7841, -0.5594, -0.7486,  0.4316, -0.0000, -0.8467, -0.9640],
          [ 0.9356,  0.9553,  0.3579, -0.4678, -0.0000, -0.6939,  0.0000],
          [-0.6782,  0.0000, -0.6991, -0.0000, -0.0000, -0.0000, -0.8023],
          [-0.0000, -0.0000, -0.7164, -0.8577,  0.2529,  0.0000, -0.1342],
          [-0.0000, -0.6964, -0.0000,  0.4933, -0.0000,  0.8302,  0.4007]]]],
       grad_fn=<TanhBackward0>)

# f(x)=x if x≥0
#      αx if x<0

In [4]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision.utils import make_grid, save_image
import matplotlib.pyplot as plt
import os

latent_dim = 100
img_size = 32
channels = 3
batch_size = 128
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
epochs = 50
lr = 0.0002
beta1 = 0.5

os.makedirs("dcgan_cifar10_output", exist_ok=True)

transform = transforms.Compose([
    transforms.Resize(img_size),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)  # Pixel: [-1, 1]
])

dataloader = torch.utils.data.DataLoader(
    torchvision.datasets.CIFAR10(root="./data", download=True, transform=transform),
    batch_size=batch_size,
    shuffle=True
)


100%|██████████| 170M/170M [00:08<00:00, 20.9MB/s]
